## Carga de datsets

Este código finaliza la preparación de los datos antes de entrenar el modelo. Sus tareas principales son:

Instalar librerías: Asegura que todas las herramientas de software necesarias estén listas.

Cargar datos: Importa los conjuntos de entrenamiento, validación y prueba.

Unificar etiquetas: Consolida las múltiples columnas de categorías en una sola lista de etiquetas por cada artículo.

Formatear para IA: Convierte los datos al formato especializado que el modelo requiere para procesarlos eficientemente.

Verificar: Confirma que todos los registros se hayan cargado correctamente en cada conjunto.

In [ ]:
# ==============================================================================
# CELDA 1: CARGA DE DATASETS (TRAIN / VAL / TEST) DESDE CSV
# ==============================================================================

import pandas as pd
import numpy as np
import torch
from datasets import Dataset

print("Instalando librerías necesarias...")
# !pip install --upgrade transformers datasets scikit-learn -q
print("¡Instalación completa!")

# Rutas a tus CSV ya separados
TRAIN_PATH = "/kaggle/input/split-dataset/train_set_expanded.csv"
VAL_PATH   = "/kaggle/input/split-dataset/val_set.csv"
TEST_PATH  = "/kaggle/input/split-dataset/test_set.csv"

TEXT_COLUMN = "text"
LABEL_COLUMNS = ["cardiovascular", "hepatorenal", "neurological", "oncological"]

def load_split(path):
    df = pd.read_csv(path)
    # Asegura tipos 0/1 en las etiquetas
    for c in LABEL_COLUMNS:
        df[c] = df[c].astype(int)
    df["labels"] = df[LABEL_COLUMNS].values.tolist()
    return df[[TEXT_COLUMN, "labels"]].copy()

try:
    train_df = load_split(TRAIN_PATH)
    val_df   = load_split(VAL_PATH)
    test_df  = load_split(TEST_PATH)
    print("✅ Datasets cargados correctamente.")
except FileNotFoundError as e:
    print(f"❌ No se encontró un archivo: {e}")
    raise

train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

print(f"Tamaño train: {len(train_dataset)}")
print(f"Tamaño val:   {len(val_dataset)}")
print(f"Tamaño test:  {len(test_dataset)}")

## Tokenizacion

Este script prepara un modelo SciBERT para una tarea de clasificación multietiqueta.

Primero, tokeniza los datasets de texto, convirtiéndolos en tensores numéricos de longitud fija para PyTorch.

Luego, instancia el modelo SciBERT y lo configura explícitamente para multi_label_classification, lo que ajusta su arquitectura para predecir múltiples categorías simultáneamente.

Finalmente, define una función para evaluar el rendimiento del modelo utilizando métricas clave como F1-score y ROC AUC.

In [ ]:
# ==============================================================================
# CELDA 2: TOKENIZACIÓN, MODELO (SCIBERT UNCASED) Y MÉTRICAS
# ==============================================================================

import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, roc_auc_score

# Nombre del modelo (SciBERT uncased)
MODEL_NAME = "allenai/scibert_scivocab_uncased"

# Se asume que LABEL_COLUMNS, train_dataset, val_dataset, test_dataset
# ya fueron creados en la Celda 1.
id2label = {i: l for i, l in enumerate(LABEL_COLUMNS)}
label2id = {l: i for i, l in enumerate(LABEL_COLUMNS)}

# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    enc = tokenizer(
        batch["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    # Etiquetas como float32 para BCEWithLogits
    enc["labels"] = [np.array(x, dtype=np.float32) for x in batch["labels"]]
    return enc

# Tokenización de los splits
train_enc = train_dataset.map(tokenize_batch, batched=True, remove_columns=train_dataset.column_names)
val_enc   = val_dataset.map(tokenize_batch,   batched=True, remove_columns=val_dataset.column_names)
test_enc  = test_dataset.map(tokenize_batch,  batched=True, remove_columns=test_dataset.column_names)

# Formato PyTorch
cols = ["input_ids", "attention_mask", "labels"]
train_enc = train_enc.with_format("torch", columns=cols)
val_enc   = val_enc.with_format("torch",   columns=cols)
test_enc  = test_enc.with_format("torch",  columns=cols)

# Modelo de clasificación multilabel
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABEL_COLUMNS),
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id
)

# Métricas (incluye F1 ponderado)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))          # sigmoid
    preds = (probs >= 0.5).astype(int)         # umbral base 0.5

    f1_micro = f1_score(labels, preds, average="micro", zero_division=0)
    f1_macro = f1_score(labels, preds, average="macro", zero_division=0)
    f1_weighted = f1_score(labels, preds, average="weighted", zero_division=0)
    try:
        auc_macro = roc_auc_score(labels, probs, average="macro")
    except ValueError:
        auc_macro = float("nan")

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "roc_auc_macro": auc_macro
    }

## entrenamiento y  selección de mejores hiperparámetros

Este script ejecuta un proceso avanzado y automatizado de optimización de hiperparámetros (HPO) utilizando la librería Optuna para encontrar la configuración de entrenamiento óptima para el modelo SciBERT. El proceso no solo busca los mejores hiperparámetros, sino que también implementa técnicas sofisticadas para manejar el desequilibrio de clases y optimizar los umbrales de decisión. Finalmente, evalúa rigurosamente el modelo campeón en el conjunto de datos de prueba y lo empaqueta para su uso futuro.

Desglose Funcional Detallado
Optimización de Hiperparámetros (HPO) con Optuna:

El núcleo del script es un bucle de optimización (study.optimize) que prueba sistemáticamente múltiples combinaciones de hiperparámetros clave (tasa de aprendizaje, tamaño de lote, decaimiento de peso, etc.) para encontrar la que maximiza el rendimiento.
El objetivo de cada "trial" (prueba) es maximizar la métrica f1_weighted en el conjunto de validación, que es una métrica robusta para problemas con desequilibrio de clases.
Manejo del Desequilibrio de Clases:

Se implementa una función de pérdida personalizada (compute_loss_with_pos_weight). Antes de entrenar, se calcula la frecuencia de cada etiqueta en el dataset de entrenamiento.
La función de pérdida (BCEWithLogitsLoss) utiliza estos cálculos para asignar un mayor peso a las clases minoritarias, forzando al modelo a prestarles más atención y evitando que se centre únicamente en las clases más comunes.
Ajuste de Umbrales de Decisión por Clase (tune_thresholds_per_class):

En la clasificación multietiqueta, un umbral de decisión estándar de 0.5 no suele ser óptimo. Esta función clave se ejecuta después de cada entrenamiento de un trial.
Para cada una de las cuatro etiquetas, busca iterativamente el umbral de probabilidad (entre 0.05 y 0.95) que maximiza el F1-score individual para esa clase en el conjunto de validación.
El rendimiento final de un trial se mide después de aplicar estos umbrales optimizados, proporcionando una evaluación mucho más precisa del verdadero potencial del modelo.
Gestión Eficiente de Recursos:

Early Stopping: Se detienen los entrenamientos de trials que no muestran mejora después de una época, ahorrando tiempo computacional.
Limpieza de Checkpoints (cleanup_callback): Se implementa un callback inteligente que, después de cada trial, elimina automáticamente la carpeta de checkpoints del modelo si este no supera al mejor modelo encontrado hasta el momento. Esto previene el consumo excesivo de espacio en disco, un problema común en HPO.
Evaluación Final y Empaquetado:

Una vez que Optuna completa todos los trials, identifica la mejor configuración (best_trial).
Carga el modelo campeón desde su checkpoint guardado y utiliza los umbrales de decisión optimizados que se calcularon para ese trial.
Realiza una evaluación final y definitiva en el conjunto de datos de prueba (test_enc), que el modelo nunca ha visto, para obtener una medida imparcial de su rendimiento en el mundo real.
Finalmente, guarda un paquete de inferencia completo que contiene el modelo entrenado, el tokenizador y el archivo best_thresholds.json. Esto permite que el modelo sea fácilmente cargado y utilizado para hacer predicciones en el futuro.

In [ ]:
# ==============================================================================
# CELDA 3: OPTUNA (HPO) + UMBRALES POR CLASE + EVALUACIÓN FINAL — CRITERIO: f1_weighted
# ==============================================================================

# Requiere que ya existan: MODEL_NAME, LABEL_COLUMNS, tokenizer, train_df, train_enc, val_enc, test_enc, compute_metrics

# !pip install optuna -q

import os
# Desactivar integraciones que bloquean (W&B) y ruido
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

import gc, json, math, optuna, numpy as np, torch, shutil # <-- NUEVO: Importamos shutil para borrar carpetas
from torch.nn import BCEWithLogitsLoss
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import f1_score, average_precision_score

# Info de dispositivo
print("CUDA disponible:", torch.cuda.is_available(), "| #GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU[0]:", torch.cuda.get_device_name(0))
    torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# En notebook, el Trainer suele usar 1 GPU. Si usas accelerate/DDP, cambia a torch.cuda.device_count().
n_gpus_eff = 1 if torch.cuda.is_available() else 0
MAX_PER_DEVICE_BS = 16  # baja a 8 si hay OOM

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tune_thresholds_per_class(probs, y_true, steps=100):
    C = probs.shape[1]
    thresholds = []
    for i in range(C):
        best_f, best_th = 0.0, 0.5
        p = probs[:, i]
        y = y_true[:, i]
        for th in np.linspace(0.05, 0.95, steps):
            f = f1_score(y, (p >= th).astype(int), zero_division=0)
            if f > best_f:
                best_f, best_th = f, th
        thresholds.append(best_th)
    thresholds = np.array(thresholds)
    preds = (probs >= thresholds).astype(int)
    return thresholds, {
        "f1_macro":     f1_score(y_true, preds, average="macro",    zero_division=0),
        "f1_micro":     f1_score(y_true, preds, average="micro",    zero_division=0),
        "f1_weighted":  f1_score(y_true, preds, average="weighted", zero_division=0),
        "auprc_macro":  average_precision_score(y_true, probs, average="macro")
    }

def build_model():
    return AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(LABEL_COLUMNS),
        problem_type="multi_label_classification",
        id2label={i: l for i, l in enumerate(LABEL_COLUMNS)},
        label2id={l: i for i, l in enumerate(LABEL_COLUMNS)}
    )

# pos_weight por desbalance (train_df['labels'])
pos_counts = np.array(train_df["labels"].tolist()).sum(axis=0)
N = len(train_df)
pos_weight = torch.tensor((N - pos_counts) / np.clip(pos_counts, 1, None), dtype=torch.float32)

def compute_loss_with_pos_weight(model, inputs, return_outputs=False,**kwargs):
    labels = inputs.pop("labels").float()
    outputs = model(**inputs)
    loss = BCEWithLogitsLoss(pos_weight=pos_weight.to(outputs.logits.device))(outputs.logits, labels)
    return (loss, outputs) if return_outputs else loss

def objective(trial):
    # Espacio de búsqueda
    learning_rate    = trial.suggest_float("learning_rate", 2.5e-5, 5e-5, log=True)
    eff_batch_size   = trial.suggest_categorical("effective_batch_size", [16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 4, 8)
    weight_decay     = trial.suggest_float("weight_decay", 0.0, 0.1) # Mantener este rango amplio
    warmup_ratio     = trial.suggest_float("warmup_ratio", 0.0, 0.1) # Acotar un poco, rara vez se necesita más

    per_device_bs = min(MAX_PER_DEVICE_BS, eff_batch_size)
    den = max(1, per_device_bs * max(1, n_gpus_eff))
    grad_accum = max(1, math.ceil(eff_batch_size / den))
    print(f"[Trial {trial.number}] per_device_bs={per_device_bs}, grad_accum={grad_accum}, eff_bs≈{per_device_bs*max(1,n_gpus_eff)*grad_accum}")

    output_dir = f"/kaggle/working/hpo_scibert_uncased/trial_{trial.number}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Guardamos la ruta del directorio para poder acceder a ella en el callback
    trial.set_user_attr("output_dir", output_dir) # <-- NUEVO: Guardamos la ruta en los atributos del trial

    model = build_model()

    args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_weighted",
        greater_is_better=True,
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_bs,
        per_device_eval_batch_size=max(per_device_bs, 32),
        gradient_accumulation_steps=grad_accum,
        weight_decay=weight_decay,
        warmup_ratio=warmup_ratio,
        logging_steps=50,
        save_total_limit=1,
        seed=42,
        fp16=torch.cuda.is_available(),
        dataloader_pin_memory=True,
        dataloader_num_workers=2, # Reducido a 2 para evitar cuellos de botella en Kaggle
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_enc,
        eval_dataset=val_enc,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )
    trainer.compute_loss = compute_loss_with_pos_weight
    trainer.train()

    val_out = trainer.predict(val_enc)
    val_probs = sigmoid(val_out.predictions)
    val_y = val_out.label_ids
    val_thresholds, val_metrics_thr = tune_thresholds_per_class(val_probs, val_y, steps=100)

    trial.set_user_attr("val_thresholds", val_thresholds.tolist())
    trial.set_user_attr("val_metrics_thr", val_metrics_thr)

    del trainer, model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return val_metrics_thr["f1_weighted"]

# ==============================================================================
# <-- INICIO: NUEVA SECCIÓN PARA GESTIÓN DE ALMACENAMIENTO -->
# ==============================================================================

def cleanup_callback(study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
    """
    Callback para limpiar los checkpoints de los trials que no son el mejor.
    Se ejecuta después de cada trial y borra la carpeta de checkpoints si
    el trial recién terminado no es el mejor hasta ahora.
    """
    # Buscamos el directorio del mejor trial hasta el momento
    try:
        best_trial_dir = study.best_trial.user_attrs.get("output_dir")
    except (AttributeError, KeyError):
        best_trial_dir = None # Aún no hay un mejor trial (p.ej. en el primer trial)

    # Buscamos el directorio del trial que acaba de terminar
    try:
        current_trial_dir = trial.user_attrs.get("output_dir")
    except (AttributeError, KeyError):
        current_trial_dir = None
        
    # Si el directorio del trial actual existe y NO es el del mejor trial, lo borramos
    if current_trial_dir and current_trial_dir != best_trial_dir and os.path.exists(current_trial_dir):
        print(f"🧹 Limpiando checkpoint del trial {trial.number} (no es el mejor). Directorio: {current_trial_dir}")
        shutil.rmtree(current_trial_dir)

# ==============================================================================
# <-- FIN: NUEVA SECCIÓN -->
# ==============================================================================

# Ejecutar estudio
study = optuna.create_study(direction="maximize", study_name="scibert_uncased_hpo",
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=2))
study.optimize(objective, n_trials=10, callbacks=[cleanup_callback]) # <-- NUEVO: Añadimos el callback

print("\nMejores hiperparámetros:", study.best_params)
print("Mejor F1_weighted (val, con thresholds):", study.best_value)

best_trial = study.best_trial
best_dir = best_trial.user_attrs["output_dir"]
best_thresholds = np.array(best_trial.user_attrs["val_thresholds"])
print("Checkpoint del mejor trial:", best_dir)
print("Umbrales del mejor trial:", best_thresholds)

# Evaluación final en TEST con umbrales del mejor trial
best_model = AutoModelForSequenceClassification.from_pretrained(
    best_dir,
    num_labels=len(LABEL_COLUMNS),
    problem_type="multi_label_classification",
    id2label={i: l for i, l in enumerate(LABEL_COLUMNS)},
    label2id={l: i for i, l in enumerate(LABEL_COLUMNS)}
)

best_args = TrainingArguments(
    output_dir="/kaggle/working/scibert_uncased_best_final",
    per_device_eval_batch_size=32,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=True,
    dataloader_num_workers=2,
    report_to="none",
)
best_trainer = Trainer(model=best_model, args=best_args, eval_dataset=test_enc)

test_out = best_trainer.predict(test_enc)
test_probs = sigmoid(test_out.predictions)
test_y = test_out.label_ids
test_preds_thr = (test_probs >= best_thresholds).astype(int)

test_metrics = {
    "f1_macro":    f1_score(test_y, test_preds_thr, average="macro",    zero_division=0),
    "f1_micro":    f1_score(test_y, test_preds_thr, average="micro",    zero_division=0),
    "f1_weighted": f1_score(test_y, test_preds_thr, average="weighted", zero_division=0),
    "auprc_macro": average_precision_score(test_y, test_probs, average="macro")
}
print("\n🧪 Métricas en TEST con umbrales ajustados (mejor trial):")
print(test_metrics)

# Guardar mejor modelo, tokenizador y umbrales
final_dir = "/kaggle/working/scibert_uncased_hpo_best"
os.makedirs(final_dir, exist_ok=True)

best_model.save_pretrained(final_dir) # <-- Guarda el modelo
tokenizer.save_pretrained(final_dir) # <-- AÑADIR ESTA LÍNEA para guardar el tokenizador

with open(os.path.join(final_dir, "best_thresholds.json"), "w") as f:
    json.dump({"labels": LABEL_COLUMNS, "thresholds": best_thresholds.tolist()}, f, indent=2) # <-- Guarda los umbrales
print(f"\n✅ Paquete de inferencia completo guardado en: {final_dir}")

## Optimización de Hiperparámetros: Resultados y Mejor Configuración
Se llevó a cabo un proceso de optimización de hiperparámetros (HPO) para encontrar la configuración óptima del modelo SciBERT en la tarea de clasificación. Se evaluaron un total de 10 combinaciones (trials) utilizando la métrica de F1-Score Ponderado (Weighted F1-Score) sobre el conjunto de validación.

Resultados Clave
Tras el análisis de los 10 trials ejecutados, el Trial 6 emergió como el de mejor rendimiento, superando a todas las demás configuraciones evaluadas.

Mejor Trial: Trial 6
Valor de la Métrica (Weighted F1-Score): 0.9623
Hiperparámetros del Mejor Modelo (Trial 6)
La configuración de hiperparámetros que produjo este resultado fue la siguiente:

Hiperparámetro	Valor	Descripción
learning_rate	4.748e-05	Tasa de aprendizaje para el optimizador AdamW.
effective_batch_size	16	Tamaño de lote efectivo para el entrenamiento.
num_train_epochs	7	Número de épocas completas de entrenamiento.
weight_decay	0.00956	Coeficiente de regularización L2 para prevenir el sobreajuste.
warmup_ratio	0.0196	Proporción de pasos de "calentamiento" para la tasa de aprendizaje.
Conclusión
El modelo final será entrenado utilizando la configuración del Trial 6, ya que ha demostrado ser la más efectiva durante la fase de experimentación y optimización. Los archivos de este modelo servirán como base para la evaluación final en el conjunto de prueba y para el despliegue de la solución.